In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
import os, sys
from pathlib import Path
BASE_DIR = Path('.').resolve().parent.parent.parent
os.chdir(BASE_DIR)
from time import time
from phm08ds.models import experiment

## Load Dataset

In [3]:
folderpath = os.path.abspath(os.getcwd()) + '/phm08ds/data/files/'
data_op_1 = pd.read_csv(folderpath + 'data_op_01.csv')
data_op_1.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_0,Sensor_1,Sensor_2,Sensor_3,Sensor_4,...,Sensor_13,Sensor_14,Sensor_15,Sensor_16,Sensor_17,Sensor_18,Sensor_19,Sensor_20,Health_state,Operational_condition
0,1,1,10.0047,0.2501,20.0,489.05,604.13,1499.45,1309.95,10.52,...,8120.83,8.6216,0.03,368,2319,100.0,28.58,17.1735,1,1
14,1,15,9.9988,0.2500,20.0,489.05,604.63,1497.87,1302.51,10.52,...,8124.37,8.6496,0.03,369,2319,100.0,28.84,17.0805,1,1
15,1,16,9.9987,0.2500,20.0,489.05,604.61,1500.47,1311.39,10.52,...,8127.07,8.6609,0.03,369,2319,100.0,28.71,17.1759,1,1
17,1,18,10.0066,0.2507,20.0,489.05,604.23,1502.57,1304.56,10.52,...,8119.39,8.6646,0.03,367,2319,100.0,28.60,17.1096,1,1
35,1,36,10.0024,0.2504,20.0,489.05,604.43,1499.85,1306.89,10.52,...,8121.73,8.6260,0.03,369,2319,100.0,28.52,17.1036,2,1


## Data preprocessing

### Get rid of informations there are not sensor readings

Wang (2008) reports Sensor 15 has importat information. However, there are no relevant informations of this sensor. The data seems to be corrupted like this:

Let's remove it from our database creating an object transformer.

In [4]:
from phm08ds.features.feature_selection import RemoveSensor

tf_remove_sensor_15 = RemoveSensor(sensors=[15])
data_op_1 = tf_remove_sensor_15.fit_transform(data_op_1)
data_op_1.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_0,Sensor_1,Sensor_2,Sensor_3,Sensor_4,...,Sensor_12,Sensor_13,Sensor_14,Sensor_16,Sensor_17,Sensor_18,Sensor_19,Sensor_20,Health_state,Operational_condition
0,1,1,10.0047,0.2501,20.0,489.05,604.13,1499.45,1309.95,10.52,...,2388.13,8120.83,8.6216,368,2319,100.0,28.58,17.1735,1,1
14,1,15,9.9988,0.2500,20.0,489.05,604.63,1497.87,1302.51,10.52,...,2388.12,8124.37,8.6496,369,2319,100.0,28.84,17.0805,1,1
15,1,16,9.9987,0.2500,20.0,489.05,604.61,1500.47,1311.39,10.52,...,2388.08,8127.07,8.6609,369,2319,100.0,28.71,17.1759,1,1
17,1,18,10.0066,0.2507,20.0,489.05,604.23,1502.57,1304.56,10.52,...,2388.06,8119.39,8.6646,367,2319,100.0,28.60,17.1096,1,1
35,1,36,10.0024,0.2504,20.0,489.05,604.43,1499.85,1306.89,10.52,...,2388.11,8121.73,8.6260,369,2319,100.0,28.52,17.1036,2,1


Before feeding to the classifier, let's remove unwanted information, such as unit, time_step and operational settings.

In [5]:
from phm08ds.features.feature_selection import RemoveInfo

tf_remove_info = RemoveInfo()

data_with_features = tf_remove_info.fit_transform(data_op_1)
data_with_features.head()

,Sensor_0,Sensor_1,Sensor_2,Sensor_3,Sensor_4,Sensor_5,Sensor_6,Sensor_7,Sensor_8,Sensor_9,...,Sensor_11,Sensor_12,Sensor_13,Sensor_14,Sensor_16,Sensor_17,Sensor_18,Sensor_19,Sensor_20,Health_state
0,489.05,604.13,1499.45,1309.95,10.52,15.49,394.88,2318.87,8770.20,1.26,...,372.15,2388.13,8120.83,8.6216,368,2319,100.0,28.58,17.1735,1
14,489.05,604.63,1497.87,1302.51,10.52,15.50,394.93,2318.90,8771.08,1.26,...,371.84,2388.12,8124.37,8.6496,369,2319,100.0,28.84,17.0805,1
15,489.05,604.61,1500.47,1311.39,10.52,15.49,394.44,2318.87,8775.86,1.26,...,372.19,2388.08,8127.07,8.6609,369,2319,100.0,28.71,17.1759,1
17,489.05,604.23,1502.57,1304.56,10.52,15.49,394.31,2318.93,8778.55,1.26,...,372.03,2388.06,8119.39,8.6646,367,2319,100.0,28.60,17.1096,1
35,489.05,604.43,1499.85,1306.89,10.52,15.49,394.36,2318.94,8771.15,1.26,...,371.77,2388.11,8121.73,8.6260,369,2319,100.0,28.52,17.1036,2


We need to normalize our data. Let's use Z-score standardization.

In [6]:
from sklearn.preprocessing import StandardScaler

tf_std_scaller = preprocessing.StandardScaler()
data_with_features_std = tf_std_scaller.fit_transform(data_with_features.drop(labels='Health_state', axis=1))
data_with_features_std

array([[-5.68434189e-14, -1.66575294e+00, -4.60782655e-01, ...,
         0.00000000e+00,  3.77665056e-01,  6.75931281e-01],
       [-5.68434189e-14, -6.08240322e-01, -7.32647265e-01, ...,
         0.00000000e+00,  2.17941314e+00, -4.02161432e-01],
       [-5.68434189e-14, -6.50540827e-01, -2.85275123e-01, ...,
         0.00000000e+00,  1.27853910e+00,  7.03753028e-01],
       ...,
       [-5.68434189e-14,  1.76058794e+00,  1.38376710e+00, ...,
         0.00000000e+00, -9.38997008e-01, -4.68238082e-01],
       [-5.68434189e-14,  2.18359299e+00,  9.98338792e-01, ...,
         0.00000000e+00, -2.04776506e+00, -1.91265047e+00],
       [-5.68434189e-14,  1.50678491e+00,  4.47726925e-01, ...,
         0.00000000e+00, -4.53910985e-01, -4.68238082e-01]])

In [14]:
labels = np.array(data_with_features['Health_state'])
labels

array([1, 1, 1, ..., 4, 4, 4])

# Classification steps

## Load Experiment model

In [7]:
from phm08ds.models import experiment

## Define classifiers and its specifications

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [9]:
# SVM
svm_linear_clf = SVC(kernel='linear')
svm_rbf_clf = SVC(kernel='rbf')
svm_poly_clf = SVC(kernel='poly')
svm_sigmoid_clf = SVC(kernel='sigmoid')

## Put all clf in a dictionary:

In [10]:
classifiers = {'SVM-Linear': svm_linear_clf, 'SVM-RBF': svm_rbf_clf, 'SVM-Poly': svm_poly_clf, 'SVM-Sigmoid': svm_sigmoid_clf}

Since we are using SVM and MLP we need to extract all power from those methods. Let's perform a Random Search to parameters optimizations.

### Hyperparameter tunning

In [11]:
from sklearn.model_selection import RandomizedSearchCV

random_search = dict((k,[]) for k in classifiers.keys())

In [12]:
param_dist_dict = {
                   'SVM-Linear': {'C': [2**i for i in range(-5,15)]},
                   'SVM-RBF': {'gamma': [2**i for i in range(-15,3)], 'C': [2**i for i in range(-5,15)]},
                   'SVM-Poly': {'gamma': [2**i for i in range(-15,3)], 'C': [2**i for i in range(-5,15)]},
                   'SVM-Sigmoid': {'gamma': [2**i for i in range(-15,3)], 'C': [2**i for i in range(-5,15)]}
                  }

In [ ]:
for clf in param_dist_dict.keys():
    start = time()
    random_search[clf] = RandomizedSearchCV(classifiers[clf], param_dist_dict[clf], cv=10,
                                            n_iter=20, verbose=5, n_jobs=100, scoring='accuracy')
    random_search[clf].fit(data_with_features_std, labels)
    print('Elapsed time:')
    print(time() - start)

## Savel results, models and pipeline to a .pkl file 

In [ ]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [ ]:
experiment.save_models(random_search, name='clf_svm')
experiment.save_pipeline(data_preprocessing)